<a href="https://colab.research.google.com/github/BilalHamdann/Code-Comment-Generator/blob/main/comment_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install transformers datasets evaluate
!pip install sacrebleu


In [12]:
dataset = load_dataset("mbpp", split="train[:1000]")

def preprocess(example):
    return {
        "input_text": example["code"],
        "target_text": example["text"]  # Treat task description as the "comment"
    }

processed_dataset = dataset.map(preprocess)


Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/codet5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [14]:
def tokenize(example):
    input_encoding = tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=256)
    target_encoding = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=64)
    input_encoding["labels"] = target_encoding["input_ids"]
    return input_encoding

tokenized_dataset = processed_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [15]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [16]:
!pip install --upgrade transformers



In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)



In [22]:
import os
os.environ["WANDB_DISABLED"] = "false"


In [24]:
!wandb login YOUR_API_KEY_HERE


Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 10, in <module>
    sys.exit(cli())
             ^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1363, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1830, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1226, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 794, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/pyth

In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

trainer.train()


/tmp/ipython-input-25-1672420396.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bilal-hamdan1998 (bilal-hamdan1998-rafik-hariri-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=126, training_loss=0.7629146878681485, metrics={'train_runtime': 1292.3708, 'train_samples_per_second': 0.78, 'train_steps_per_second': 0.097, 'total_flos': 68212267941888.0, 'train_loss': 0.7629146878681485, 'epoch': 3.0})

In [26]:
test_samples = eval_dataset.select(range(5))

for sample in test_samples:
    input_ids = tokenizer(sample["input_text"], return_tensors="pt", truncation=True, max_length=256).input_ids
    generated_ids = model.generate(input_ids, max_length=64, num_beams=4, temperature=0.7)
    predicted_comment = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print("📌 Code:\n", sample['input_text'])
    print("📝 Ground Truth Comment:\n", sample['target_text'])
    print("🤖 Generated Comment:\n", predicted_comment)
    print("=" * 80)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📌 Code:
 def check_none(test_tup):
  res = any(map(lambda ele: ele is None, test_tup))
  return (res) 
📝 Ground Truth Comment:
 Write a function to check if the given tuple has any none value or not.
🤖 Generated Comment:
 Write a python function to check if any of the elements in a given list.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📌 Code:
 def camel_to_snake(text):
        import re
        str1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
        return re.sub('([a-z0-9])([A-Z])', r'\1_\2', str1).lower()
📝 Ground Truth Comment:
 Write a function to convert camel case string to snake case string.
🤖 Generated Comment:
 Write a python function to convert a string to camel case.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📌 Code:
 def find_triplet_array(A, arr_size, sum): 
	for i in range( 0, arr_size-2): 
		for j in range(i + 1, arr_size-1): 
			for k in range(j + 1, arr_size): 
				if A[i] + A[j] + A[k] == sum: 
					return  A[i],A[j],A[k] 
					return True
	return False
📝 Ground Truth Comment:
 Write a function to find if there is a triplet in the array whose sum is equal to a given value.
🤖 Generated Comment:
 Write a python function to find the triplet of a given array.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📌 Code:
 def mutiple_tuple(nums):
    temp = list(nums)
    product = 1 
    for x in temp:
        product *= x
    return product
📝 Ground Truth Comment:
 Write a python function to calculate the product of all the numbers of a given tuple.
🤖 Generated Comment:
 Write a python function to find the product of mutiple tuples.
📌 Code:
 def add_dict_to_tuple(test_tup, test_dict):
  test_tup = list(test_tup)
  test_tup.append(test_dict)
  test_tup = tuple(test_tup)
  return (test_tup) 
📝 Ground Truth Comment:
 Write a function to add a dictionary to the tuple.
🤖 Generated Comment:
 Write a function to add a dict to a tuple.


In [28]:
import evaluate

bleu = evaluate.load("sacrebleu")

# Use all samples in your eval_dataset safely, no out-of-range errors
n = len(eval_dataset)
print(f"Evaluating BLEU on {n} samples...")

preds = []
refs = []

for sample in eval_dataset:
    input_ids = tokenizer(
        sample["input_text"], return_tensors="pt", truncation=True, max_length=256
    ).input_ids
    output_ids = model.generate(input_ids, max_length=64, num_beams=4)
    predicted_comment = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    preds.append(predicted_comment)
    refs.append([sample["target_text"]])

score = bleu.compute(predictions=preds, references=refs)
print(f"🔍 BLEU Score: {score['score']:.2f}")



Evaluating BLEU on 38 samples...
🔍 BLEU Score: 31.56


In [29]:
trainer.save_model("comment-generator-model")
tokenizer.save_pretrained("comment-generator-model")


('comment-generator-model/tokenizer_config.json',
 'comment-generator-model/special_tokens_map.json',
 'comment-generator-model/vocab.json',
 'comment-generator-model/merges.txt',
 'comment-generator-model/added_tokens.json',
 'comment-generator-model/tokenizer.json')